In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [40]:
import glob
import random 
import time

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from torch.autograd import Variable
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

In [46]:
transform = transforms.Compose([transforms.ToTensor()])

In [47]:
dataset = datasets.ImageFolder('./rps-cv-images/', transform=transform)

In [49]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [52]:
# Looping through it, get a batch on each loop 
for images, labels in dataloader:
    pass

# Get one batch
images, labels = next(iter(dataloader))
print(images.shape,labels.shape)

torch.Size([32, 3, 200, 300]) torch.Size([32])
